In [18]:
from transformers import BertTokenizer, EncoderDecoderModel, Seq2SeqTrainer, Seq2SeqTrainingArguments
from torch.utils.data import Dataset, DataLoader
import pandas as pd

model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-uncased', 'bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertLMHeadModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if

In [20]:
class SentenceQuestionDataset(Dataset):

    def __init__(self, tokenizer, sentences, questions, max_len=128):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.questions = questions
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        sentence = self.sentences[idx]
        question = self.questions[idx]

        inputs = self.tokenizer.encode_plus(sentence, padding='max_length', max_length=self.max_len, return_tensors='pt')
        outputs = self.tokenizer.encode_plus(question, padding='max_length', max_length=self.max_len, return_tensors='pt')

        return {
            'input_ids': inputs['input_ids'].flatten(),
            'attention_mask': inputs['attention_mask'].flatten(),
            'decoder_input_ids': outputs['input_ids'].flatten(),
            'decoder_attention_mask': outputs['attention_mask'].flatten(),
            'labels': outputs['input_ids'].flatten()
        }

In [21]:
# Load the data
df = pd.read_csv('train_CSV.csv', header=None, encoding='utf-8', sep=';')
df = df.dropna()
sentences = df[0].values.tolist()
questions = df[1].values.tolist()

# Create the dataset
dataset = SentenceQuestionDataset(tokenizer, sentences, questions, max_len=128)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

print(len(sentences))
print(len(questions))
print(sentences[0])
print(questions[0])

10829
10829
I am a big one or a little one 
 am I a big one or a little one ?


In [22]:
training_args = Seq2SeqTrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=24,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
)

In [24]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

  0%|          | 0/2260 [00:00<?, ?it/s]

RuntimeError: Placeholder storage has not been allocated on MPS device!

In [ ]:
def generate_question(sentence):
    inputs = tokenizer.encode(sentence, return_tensors='pt')
    inputs = inputs.to(device)

    question_ids = model.generate(inputs, num_beams=5, max_length=20, early_stopping=True)
    question = tokenizer.decode(question_ids[0], skip_special_tokens=True)

    return question

generate_question("The cat is there")